<a href="https://colab.research.google.com/github/Emmanuelara1/Life_Analytics/blob/main/bio_image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from skimage import measure, filters
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from skimage import exposure
from scipy import ndimage


In [ ]:

model = load_model('path_to_model.h5')

def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_filtered = cv2.bilateralFilter(img, 9, 75, 75)


    _, img_thresh = cv2.threshold(img_filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


    img_normalized = exposure.equalize_hist(img_thresh)

    return img_normalized

def segment_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    img_thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    kernel = np.ones((5,5), np.uint8)
    img_dilated = cv2.dilate(img_thresh, kernel, iterations=2)
    img_eroded = cv2.erode(img_dilated, kernel, iterations=1)

    return img_eroded

def classify_image(image_path):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (224, 224))
    img_array = np.expand_dims(img_resized, axis=0) / 255.0


    prediction = model.predict(img_array)
    label = np.argmax(prediction, axis=1)

    return label, prediction

def process_image(image_path):
    preprocessed_image = preprocess_image(image_path)
    segmented_image = segment_image(image_path)
    label, prediction = classify_image(image_path)

    return preprocessed_image, segmented_image, label, prediction

def save_result(image, output_path):

    cv2.imwrite(output_path, image)

def display_image(image):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
